In [ ]:
from importlib import reload
from msclap.dataset import get_data_loader
from tqdm import tqdm
import pandas as pd
import torch

from msclap import CLAP
import gc
torch.cuda.empty_cache()
gc.collect()

In [ ]:
batch_size = 8
# train_csv_path = '/content/drive/MyDrive/Practice/DeepLearning/T2A/CLAP/dataset_tta/maple_svn/csv/train.csv'
# val_csv_path = '/content/drive/MyDrive/Practice/DeepLearning/T2A/CLAP/dataset_tta/maple_svn/csv/val.csv'

# origin_train = pd.read_csv(train_csv_path)
# origin_val = pd.read_csv(val_csv_path)
# origin_train

# train_loader = get_data_loader(csv_path=train_csv_path, batch_size=batch_size, mode='train')
# val_loader = get_data_loader(csv_path=val_csv_path, batch_size=batch_size, mode='val')


In [6]:
for audio_files, class_labels in train_loader:
    print(audio_files)
    print(class_labels)
    break

# from IPython.display import display, Audio
# audio_files = [
#     '/content/drive/MyDrive/Practice/DeepLearning/T2A/CLAP/dataset_tta/maple_svn/audio/Skill/0000097_Hit.mp3',
#     '/content/drive/MyDrive/Practice/DeepLearning/T2A/CLAP/dataset_tta/maple_svn/audio/Skill/0000097_Use.mp3',
#     '/content/drive/MyDrive/Practice/DeepLearning/T2A/CLAP/dataset_tta/maple_svn/audio/Skill/0000099_Hit.mp3',
#     '/content/drive/MyDrive/Practice/DeepLearning/T2A/CLAP/dataset_tta/maple_svn/audio/Skill/0000099_Use.mp3',
#     '/content/drive/MyDrive/Practice/DeepLearning/T2A/CLAP/dataset_tta/maple_svn/audio/Skill/0000100_Hit.mp3',
# ]
# display(Audio(audio_files[2]))

# class_labels = [
#     'This Sound is class of sharp blade',
#     'This Sound is class of blunt blade',
#     'fire',
#     'blunt weapon',
#     'ax',
# ]

['/content/drive/MyDrive/Practice/DeepLearning/T2A/CLAP/dataset_tta/maple_svn/audio/Skill_44100/65121052_Use.mp3', '/content/drive/MyDrive/Practice/DeepLearning/T2A/CLAP/dataset_tta/maple_svn/audio/Skill_44100/135001000_Hit.mp3', '/content/drive/MyDrive/Practice/DeepLearning/T2A/CLAP/dataset_tta/maple_svn/audio/Skill_44100/400001064_Hit.mp3', '/content/drive/MyDrive/Practice/DeepLearning/T2A/CLAP/dataset_tta/maple_svn/audio/Skill_44100/63141012_Hit.mp3', '/content/drive/MyDrive/Practice/DeepLearning/T2A/CLAP/dataset_tta/maple_svn/audio/Skill_44100/31211003_Use.mp3', '/content/drive/MyDrive/Practice/DeepLearning/T2A/CLAP/dataset_tta/maple_svn/audio/Skill_44100/80003538_Use.mp3', '/content/drive/MyDrive/Practice/DeepLearning/T2A/CLAP/dataset_tta/maple_svn/audio/Skill_44100/35141002_Hit.mp3', '/content/drive/MyDrive/Practice/DeepLearning/T2A/CLAP/dataset_tta/maple_svn/audio/Skill_44100/22110023_Use.mp3']
['The sharp, forceful sound of a magical weapon, like a wand or staff, conjuring the 

In [ ]:

# Load model (Choose between versions '2022' or '2023')
# The model weight will be downloaded automatically if `model_fp` is not specified
clap_model = CLAP(version = '2023', use_cuda=True)

# # Extract text embeddings
# text_embeddings = clap_model.get_text_embeddings(class_labels = class_labels)

# # Extract audio embeddings
# audio_embeddings = clap_model.get_audio_embeddings(audio_files=audio_files[:1])

# # Compute similarity between audio and text embeddings
# similarities = clap_model.compute_similarity(audio_embeddings, text_embeddings)
# print(f"class_labels : {class_labels}")
# similarities

In [ ]:
audio_encoder_finetune = False
caption_encoder_finetune = False
projection_layer_finetune = True

if audio_encoder_finetune:
    clap_model.clap.audio_encoder.eval()
    for n, p in clap_model.clap.audio_encoder.named_parameters():
        p.requires_grad = True
else:
    clap_model.clap.audio_encoder.train()
    for n, p in clap_model.clap.audio_encoder.named_parameters():
        p.requires_grad = False
    

if caption_encoder_finetune:
    clap_model.clap.caption_encoder.eval()
    for n, p in clap_model.clap.caption_encoder.named_parameters():
        p.requires_grad = True
else:
    clap_model.clap.caption_encoder.train()
    for n, p in clap_model.clap.caption_encoder.named_parameters():
        p.requires_grad = False
    

In [53]:
# clap_model.clap.eval()
# clap_model.clap.audio_encoder.eval()
# clap_model.clap.caption_encoder.eval()
torch.cuda.empty_cache()
gc.collect()

52

In [ ]:
audio_sample_tensor = torch.randn(16,600)
audio_sample_tensor = audio_sample_tensor.to(0)

audio_embed, _ = clap_model.clap.audio_encoder(audio_sample_tensor)
audio_embed.shape

# print(clap_model.clap.audio_encoder(sample_tensor)[0].shape) # (B, embed_size)
# print(clap_model.clap.audio_encoder(sample_tensor)[1].shape) # (B, class_num)

torch.Size([16, 1024])

In [ ]:
# sample_tensor = [
#                  'this sound is of the bladed weapon',
#                  'this sound is of the blunt weapon',
#                  ]
batch_size = 8
text_len = 25
texts = torch.randint(1, 300, size=(batch_size, text_len))
texts = texts.to(0)
attention_mask = torch.ones(batch_size, text_len)
text_batch = {
    'input_ids': texts,
    'attention_mask': attention_mask,
}

caption_embed = clap_model.clap.caption_encoder(text_batch)
caption_embed.shape

# print(clap_model.clap.caption_encoder(sample_tensor)[0].shape) # (B, embed_size)
# print(clap_model.clap.caption_encoder(sample_tensor)[1].shape) # (B, class_num)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [68]:
text_sample_tensor['input_ids']

IndexError: too many indices for tensor of dimension 2

### audio encoder config

- enable repeat mode = True로 하면 알아서 반복 채워넣나? -> 안됨

In [ ]:
## config
# audio encoder config
audio_encoder_config = clap_model.clap.audio_encoder.base.htsat.config
audio_encoder_config.enable_repeat_mode
# help(audio_encoder_config)

# caption encoder config
caption_encoder_config = clap_model.clap.caption_encoder.base.config
caption_encoder_config
# caption_encoder_config.enable_repeat_mode

GPT2Config {
  "_attn_implementation_autoset": true,
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.48.3",
  "use_cache": true,
  "vocab_size": 50257
}

In [ ]:
def train_epoch(model, train_loader, optimizer, lr_scheduler, step):
    loss_meter = AvgMeter()
    tqdm_object = tqdm(train_loader, total=len(train_loader))
    for batch in tqdm_object:
        batch = {k: v.to(CFG.device) for k, v in batch.items() if k != "caption"}
        loss = model(batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if step == "batch":
            lr_scheduler.step()

        count = batch["image"].size(0)
        loss_meter.update(loss.item(), count)

        tqdm_object.set_postfix(train_loss=loss_meter.avg, lr=get_lr(optimizer))
    return loss_meter
